# 1. 라이브러리 임포트

In [1]:
import pandas as pd
import numpy as np

# 2. 데이터 로드 및 추가 수정된 데이터 통합

In [2]:
# 데이터 로드
df = pd.read_csv("Total_Data2.csv")
print(df.shape)
df.head(1)

(2525, 53)


,index,url,title,post_date,검색어,브랜드 명,content,content_cnt,content_line,quote,...,구매후기 키워드,한달 키워드,두달 키워드,세달 키워드,WEB_광고 분류1,WEB_광고 분류2,광고 분류1,광고 분류2,jsonFile,context_img
0,0,https://blog.naver.com/PostView.naver?blogId=e...,무선 무릎찜질기 메이크맨 무르비2 사용 후기,20220124,무르비2 리뷰,메이크맨,저는 여름보다 유달리 겨울 추위를 잘 타는 체질이라 ... 히터가 작동되는 사무실에...,2435,22,['편리한 무선 온열찜질기진동 마사지 기능옥이 들어있어 원적외선 효과'],...,0,0,0,0,0,0,1,2,annotation/2/0_class2.json,NaN


## 2.1 추가 데이터 로드

In [3]:
df1 = pd.read_csv("result.csv")
df1.columns = ["url","new_content","coupan.ng 키워드"]

In [4]:
df.shape, df1.shape

((2525, 53), (3080, 3))

## 2.2 데이터 병합
<br>
데이터 통합 전에 동일한 content에 대해서 컬럼 삭제 후 merge 진행을 합니다.

In [5]:
df = df.drop("content",axis=1)

In [6]:
df2 = df.merge(df1,on="url",how="inner")
df2.shape

(2566, 54)

## 2.3 중복된 데이터 제거
<br>
병합 후 중복된 데이터 찾아 제거

In [7]:
gr1 = df2["url"].value_counts().reset_index()

In [8]:
gr2 = df2.set_index("url").loc[list(gr1[gr1["url"] >1]["index"])]

In [9]:
gr2.drop("index",axis=1,inplace=True)

In [10]:
gr2 = gr2.drop_duplicates()
gr2.shape

(48, 52)

In [11]:
gr2[["광고 분류1","광고 분류2"]]["광고 분류1"].value_counts()

0    28
1    20
Name: 광고 분류1, dtype: int64

In [12]:
# 중복된 데이터 index 정보를 B로 저장
B = set(list(gr1[gr1["url"] >1]["index"]))

In [13]:
# 전체 index에 대해서 정보를 A로 저장
A = set(df2["url"].unique())

In [14]:
# 차집합으로 유니크한 인덱스 번호 추출
A = A - B

In [15]:
# 중복되지 않은 정보만 추출
df2 = df2.set_index("url").loc[A]

C:\Users\User\AppData\Local\Temp\ipykernel_1664\2225915705.py:2: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  df2 = df2.set_index("url").loc[A]


In [16]:
df2 = df2.reset_index()

# 3. Feature 추출

1. Context 데이터 뽑기

In [17]:
df2[df2["new_content"].str.contains('제공|대여|무상|지원|업체|원고료|소정료|취재비|제품|서비스|체험단|제작비|제작|운고료|삭성|제험단|제삭|체삭|협찬|부터|밥아|받아|선정|선점|제금|반아|세작|협진|받이')]["광고 분류2"].value_counts() 

0    1416
2     346
1     141
Name: 광고 분류2, dtype: int64

In [18]:
df2[df2["new_content"].str.contains('원고료|체험단|제작비')]["광고 분류2"].value_counts()

2    78
0    16
1     2
Name: 광고 분류2, dtype: int64

In [28]:
kk = '파트너스|수수료'
df2[df2["new_content"].str.contains(kk)]["광고 분류2"].value_counts()

1    79
2     9
Name: 광고 분류2, dtype: int64

In [20]:
def print_url(x):
    print(x[0])

df2[(df2["new_content"].str.contains(kk)) & (df2["광고 분류2"] ==0)][["url"]].apply(lambda x: print_url(x),axis=1)

https://blog.naver.com/PostView.naver?blogId=ali7548&logNo=222043881792
https://blog.naver.com/PostView.naver?blogId=alswl970921&logNo=222514247016
https://blog.naver.com/PostView.naver?blogId=amfih&logNo=222569338425
https://blog.naver.com/PostView.naver?blogId=parkyouna8&logNo=222257194709
https://blog.naver.com/PostView.naver?blogId=hjh100311&logNo=222651913675
https://blog.naver.com/PostView.naver?blogId=tnqls2632&logNo=222315959450
https://blog.naver.com/PostView.naver?blogId=jeongyonghan&logNo=222339087539
https://blog.naver.com/PostView.naver?blogId=su0516&logNo=222514677445
https://blog.naver.com/PostView.naver?blogId=yskaygo&logNo=222607941718
https://blog.naver.com/PostView.naver?blogId=singry_y&logNo=222396201986
https://blog.naver.com/PostView.naver?blogId=hoiroi&logNo=222644693491
https://blog.naver.com/PostView.naver?blogId=snowballball&logNo=222196171943


63      None
214     None
303     None
453     None
512     None
1094    None
1502    None
1537    None
1617    None
1650    None
2317    None
2409    None
dtype: object

In [26]:
v_m = list(df2[(df2["new_content"].str.contains(kk)) & (df2["광고 분류2"] ==0)].index)

In [22]:
for k in v_m:
    df2.loc[k,"광고 분류2"] = 1
    df2.loc[k,"광고 분류1"] = 1

In [29]:
df2[(df2["new_content"].str.contains(kk)) & (df2["광고 분류2"] ==0)][["url"]].apply(lambda x: print_url(x),axis=1)

,url


In [32]:
# kk = '없는'
# df2[df2["new_content"].str.contains(kk)]["광고 분류2"].value_counts()

0    470
2    170
1     39
Name: 광고 분류2, dtype: int64

In [34]:
df2.loc[1381,"광고 분류2"] = 2
df2.loc[1381,"광고 분류1"] = 1

In [35]:
df2.loc[2426,"광고 분류2"] = 2
df2.loc[2426,"광고 분류1"] = 1

In [36]:
df2.loc[1286,"광고 분류2"] = 1
df2.loc[1286,"광고 분류1"] = 1

In [87]:
df2.loc[df2['index'] == 121, "광고 분류2"] =2
df2.loc[df2['index'] == 121, "광고 분류1"] =1

df2.loc[df2['index'] == 825, "광고 분류2"] =2
df2.loc[df2['index'] == 825, "광고 분류1"] =1


df2.loc[df2['index'] == 2287, "광고 분류2"] =2
df2.loc[df2['index'] == 2287, "광고 분류1"] =1

df2.loc[df2['index'] == 2221, "광고 분류2"] =1
df2.loc[df2['index'] == 2221, "광고 분류1"] =1

df2.loc[df2['index'] == 1057, "광고 분류2"] =0
df2.loc[df2['index'] == 1057, "광고 분류1"] =0

df2.loc[df2['index'] == 2523, "광고 분류2"] =0
df2.loc[df2['index'] == 2523, "광고 분류1"] =0

---

In [41]:
df2.columns = ['url', 'index', 'title', 'post_date', '검색어', '브랜드 명', 'content_cnt',
       'content_line', 'quote', 'quote_cnt', 'first_img', 'last_img',
       'img_cnt', '솔직 빈도 수', '비교 빈도 수', 'ㅋ 빈도 수', 'ㅋㅋ 빈도 수', 'ㅋㅋㅋ 빈도 수',
       'ㅋㅋㅋㅋ 빈도 수', '... 빈도 수', '.... 빈도 수', 'ㅜ 빈도 수', 'ㅜㅜ 빈도 수', 'ㅜㅜㅜ 빈도 수',
       'ㅜㅜㅜㅜ 빈도 수', 'ㅠ 빈도 수', 'ㅠㅠ 빈도 수', 'ㅠㅠㅠ 빈도 수', 'ㅠㅠㅠㅠ 빈도 수', '장점 빈도 수',
       '단점 빈도 수', '비속어 빈도 수', '허락 키워드', '내돈내산 키워드', '리얼후기 키워드', '협찬 키워드',
       '체험단 키워드', 'coupa.ng 키워드', '< 키워드', '> 키워드', '♡ 키워드', '♥ 키워드',
       '구매후기 키워드', '한달 키워드', '두달 키워드', '세달 키워드', 'WEB_광고 분류1', 'WEB_광고 분류2',
       '광고 분류1', '광고 분류2', 'jsonFile', 'context_img', 'content',
       'coupan.ng 키워드']

In [42]:
df2 = df2[['url', 'index', 'title','content', 'post_date', '검색어', '브랜드 명', 'content_cnt',
       'content_line', 'quote', 'quote_cnt', 'first_img', 'last_img',
       'img_cnt', '솔직 빈도 수', '비교 빈도 수', 'ㅋ 빈도 수', 'ㅋㅋ 빈도 수', 'ㅋㅋㅋ 빈도 수',
       'ㅋㅋㅋㅋ 빈도 수', '... 빈도 수', '.... 빈도 수', 'ㅜ 빈도 수', 'ㅜㅜ 빈도 수', 'ㅜㅜㅜ 빈도 수',
       'ㅜㅜㅜㅜ 빈도 수', 'ㅠ 빈도 수', 'ㅠㅠ 빈도 수', 'ㅠㅠㅠ 빈도 수', 'ㅠㅠㅠㅠ 빈도 수', '장점 빈도 수',
       '단점 빈도 수', '비속어 빈도 수', '허락 키워드', '내돈내산 키워드', '리얼후기 키워드', '협찬 키워드',
       '체험단 키워드', 'coupa.ng 키워드', '< 키워드', '> 키워드', '♡ 키워드', '♥ 키워드',
       '구매후기 키워드', '한달 키워드', '두달 키워드', '세달 키워드', 'WEB_광고 분류1', 'WEB_광고 분류2',
       '광고 분류1', '광고 분류2', 'jsonFile', 'context_img','coupan.ng 키워드']]

In [43]:
df2.to_csv("Total_Data_3(1).csv",index=False,encoding="utf-8-sig")

1. content에서 feature 생성하기


In [39]:
df2.shape

(2470, 54)

In [44]:
origin = df2.copy()

In [45]:
#  content에서 쿠팡 파트너스 특징 추출
kk = '파트너스|수수료'
df2["쿠팡키워드"] = 0
df2.loc[df2["content"].str.contains(kk),"쿠팡키워드"]=1

C:\Users\User\AppData\Local\Temp\ipykernel_1664\885172291.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2["쿠팡키워드"] = 0


In [46]:
#  context에서 일반적 광고 특징 추출
kk = '원고료|체험단|제작비'
df2["광고키워드"] = 0

df2.loc[df2["content"].str.contains(kk),"광고키워드"]=1

C:\Users\User\AppData\Local\Temp\ipykernel_1664\3844507292.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2["광고키워드"] = 0


In [47]:
# 광고성 글 제외 feature
kk = '대가 없|없는|제외|빼고|양심'
df2["솔직키워드1"] = 0

df2.loc[df2["content"].str.contains(kk),"솔직키워드1"]=1

C:\Users\User\AppData\Local\Temp\ipykernel_1664\2803794097.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2["솔직키워드1"] = 0


In [48]:
kk = '솔직'
df2["솔직키워드2"] = 0

df2.loc[df2["content"].str.contains(kk),"솔직키워드2"]=1

C:\Users\User\AppData\Local\Temp\ipykernel_1664\3313139176.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2["솔직키워드2"] = 0


---

2. 이미지에서 OCR 결과 전처리

In [49]:
df2["context_img"].fillna("",inplace=True)

C:\Users\User\AppData\Local\Temp\ipykernel_1664\1788420777.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2["context_img"].fillna("",inplace=True)


* 협찬 등

In [50]:
kk1 = '체험|대여|무상|지원|업체|원고료|소정|제작'
kk2 = '체험|대여|무상|지원|업체|원고료|소정|협찬|원고|서비스|제작|작성'
kk3 = '제공|대여|무상|지원|업체|원고료|소정료|취재비|제품|서비스|체험단|제작비|제작|운고료|삭성|제험단|제삭|체삭|협찬|부터|밥아|받아|선정|선점|제금|반아|세작|협진|받이'

In [51]:
df2["이미지광고키워드1"] = 0
df2.loc[df2["context_img"].str.contains(kk1),"이미지광고키워드1"]=1

df2["이미지광고키워드2"] = 0
df2.loc[df2["context_img"].str.contains(kk2),"이미지광고키워드2"]=1

df2["이미지광고키워드3"] = 0
df2.loc[df2["context_img"].str.contains(kk3),"이미지광고키워드3"]=1

C:\Users\User\AppData\Local\Temp\ipykernel_1664\1182314735.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2["이미지광고키워드1"] = 0
C:\Users\User\AppData\Local\Temp\ipykernel_1664\1182314735.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2["이미지광고키워드2"] = 0
C:\Users\User\AppData\Local\Temp\ipykernel_1664\1182314735.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentati

* 쿠팡 파트너스

In [52]:
kk = '파트너스|쿠팡|일정액|수수료|수수|수익|쿠광|일원|일워'

df2["이미지쿠팡키워드1"] = 0
df2.loc[df2["context_img"].str.contains(kk),"이미지쿠팡키워드1"]=1

C:\Users\User\AppData\Local\Temp\ipykernel_1664\1585849598.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2["이미지쿠팡키워드1"] = 0


* 광고성 글 제외 feature

In [53]:
kk = '내돈내산|사비|직접|구매|일정액'

df2["이미지순수키워드1"] = 0
df2.loc[df2["context_img"].str.contains(kk),"이미지순수키워드1"]=1

C:\Users\User\AppData\Local\Temp\ipykernel_1664\2148595445.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2["이미지순수키워드1"] = 0


In [54]:
df2["이미지없음"] = 0
df2.loc[df2["last_img"].isnull(), "이미지없음"] = 1

C:\Users\User\AppData\Local\Temp\ipykernel_1664\2310467518.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2["이미지없음"] = 0


In [55]:
df2["이미지글없음"] = 0
df2.loc[df2['context_img']=="", "이미지글없음"]= 1

C:\Users\User\AppData\Local\Temp\ipykernel_1664\2923164397.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2["이미지글없음"] = 0


# 4. 데이터 저장

In [91]:
df2.to_csv("Total_Data_4(1).csv",index=False,encoding="utf-8-sig")